In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import time
import glob
from selenium import webdriver
import time
import pymssql #运行此程序前，需要安装pymssql
import pandas as pd
import key

from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, INT,Text

In [30]:
def ACS_updatedownload(key):
    conn = pymssql.connect(host=key.sql.host, port=key.sql.port ,user=key.sql.username, password=key.sql.password)
    query = "select doi from [BP_Extension].[dbo].[ACS_Edsug]"
    df_utnull = pd.read_sql(query,con=conn)
    lst_doi = list(df_utnull['doi'])
    # 提交之前的操作，如果之前已经执行多次的execute，那么就都进行提交
    # 关闭connection对象
    conn.close()
    # 创建connection连接
    tag = 1
    #lst_dic = []

    tag = 1
    #lst_doi = []
    lst_dic = []
    while(tag):
        print(tag)
        u = "https://pubs.acs.org/editorschoice/?widgetSort=EditorsChoiceDate&widgetFilter=&widgetPage=" + str(tag-1)
        r = requests.get(u)
        soupi = BeautifulSoup(r.text, 'lxml')

        for i in range(len(soupi.find_all('div',class_ = 'issue-item clearfix'))):
            #print(i)

            dic = {}
            contenti = str(soupi.find_all('div',class_ = 'issue-item clearfix')[i])
            try:
                dic['img_paper'] = re.match('[\s\S]*"Article figure" class="lazy" data-src="([\s\S]*?)"/><noscript>[\s\S]*',contenti).group(1)
            except:
                1
            try:
                dic['img_journal'] = re.match('[\s\S]*alt="Journal logo" src="([\s\S]*)"/></noscript>[\s\S]*',contenti).group(1)
            except:
                1
            dic['doi'] = re.match('[\s\S]*"issue-item_title"><a href="/([\s\S]*?)" title="[\s\S]*',contenti).group(1).replace('doi/','')
            dic['title'] = soupi.find_all('h5',class_ ='issue-item_title')[i].text
            dic['authors'] = soupi.find_all('ul',class_ ='rlist--inline loa mobile-authors issue-item_loa')[i].text.replace('\xa0',' ')
            dic['journal_full'] = soupi.find_all('div',class_ ='issue-item_info_list')[i].text
            dic['journal'] = re.match('[\s\S]*issue-item_jour-name"><i>([\s\S]*)</i></span>[\s\S]*',contenti,re.M).group(1)

            dic['year'] = re.match('[\s\S]*<span class="issue-item_yearpubdate"> <span>([\s\S]*?)</span></span>[\s\S]*',contenti).group(1)
            dic['paper_type'] = re.match('[\s\S]*<span class="issue-item_type"> \(([\s\S]*?)\)</span>[\s\S]*',contenti).group(1)
            dic['pubulicationdata'] = soupi.find_all('span',class_='issue-item_pubdate')[i+i].text
            try:
                dic['choicedate'] = re.match('[\s\S]*ACS Editors\' Choice Date</span><span class="date-separator">:</span>([\s\S]*?)</span>[\s\S]*',contenti).group(1)
            except:
                1
            key = soupi.find_all('ul',class_='issue-item_buttons-list')[i].find_all('li')[0].a['title']
            dic[key] = soupi.find_all('ul',class_='issue-item_buttons-list')[i].find_all('li')[0].a['href']
            try:
                dic['fulltext_link'] = re.match('[\s\S]*<li><a href="([\s\S]*?)" title="Full text">[\s\S]*',contenti).group(1)
            except:
                1
            dic['pdf_link'] = re.match('[\s\S]*<li><a href="([\s\S]*?)" title="PDF"[\s\S]*',contenti).group(1)
            try:
                dic['abstract'] = soupi.find_all('div',class_='issue-item_footer')[i].find('div').find('div').text
            except:
                1
            #print(dic['title'])
            if dic['doi'] not in lst_doi:
                lst_doi.append(dic['doi'])
                lst_dic.append(dic)
            else:
                tag = -1
                break
        tag = tag + 1
        print('记录数',len(lst_dic))
    print('爬取完毕')
    print(len(lst_dic))
    return(lst_dic)
def ACSprocess(lst_dic):
    '''
    输入数据字典列表
    '''
    if len(lst_dic) >0:
        print('开始数据处理')
        df = pd.DataFrame(lst_dic)
        df['img_paper_fix'] = 'https://pubs.acs.org' + df['img_paper']
        df['img_journal_fix'] = 'https://pubs.acs.org' + df['img_journal']
        df['doi'] = df['doi'].str.replace('doi/','')

        df['Abstract_fix'] = 'https://pubs.acs.org' + df['Abstract']
        df['Abstract_link'] = df['Abstract']
        df = df.drop('Abstract',axis =1)
        df['fulltext_link_fix'] = 'https://pubs.acs.org' + df['fulltext_link']
        df['pdf_link_fix'] = 'https://pubs.acs.org' + df['pdf_link']
        try:
            df['First_page_fix'] = 'https://pubs.acs.org' + df['First Page']
        except:
            1
        try:
            df['Fulltext_fix'] = 'https://pubs.acs.org' + df['Full text']
        except:
            1
        print('数据处理完成')
        return df
    else:
        print('无处理记录')
        return pd.DataFrame()

def CASuploadData(df,key):
    conn = create_engine('mssql+pymssql://'+ key.sql.username +':' + key.sql.password+ '@' + key.sql.host+ '/BP_Extension')
    dtypedict = {
        'img_paper' : NVARCHAR(length=255),
        'img_journal' : NVARCHAR(length=255),
        'doi': NVARCHAR(length=255),
        'title': Text(),
        'authors': Text(),
        'journal_full': NVARCHAR(length=255),
        'Abstract': NVARCHAR(length=255),
        'fulltext_link': NVARCHAR(length=255),
        'pdf_link': NVARCHAR(length=255),
        'PDF': NVARCHAR(length=255),
        '[First Page]': NVARCHAR(length=255),
        '[Full text]': NVARCHAR(length=255),
        'img_paper_fix': NVARCHAR(length=255),
        'img_journal_fix': NVARCHAR(length=255),
        'Abstract_fix': NVARCHAR(length=255),
        'fulltext_link_fix': NVARCHAR(length=255),
        'pdf_link_fix': NVARCHAR(length=255),
        'First_page_fix': NVARCHAR(length=255),
        'Fulltext_fix': NVARCHAR(length=255),
        'ACS_tag': INT(),
        'UT': NVARCHAR(length=255)
    }
    df.to_sql(name='ACS_Edsug', con=conn, if_exists='replace', index=False, dtype=dtypedict)
    conn.dispose()

def QueryGener_ACS(key):
    conn = pymssql.connect(host=key.sql.host, port=key.sql.port ,user=key.sql.username, password=key.sql.password)
    query = "select doi,title,UT from [BP_Extension].[dbo].[ACS_Edsug] where UT = '' or UT = 'nan'"
    df = pd.read_sql(query,con=conn)
    conn.close()
    query = ''
    for i in range(len(df)):
        queryi = 'DO = (' + df.iloc[i]['doi'] + ')'
        if i == 0:
            query = queryi
            continue
        query = query + ' OR ' + queryi

    newfile = './Query_ACS.txt'
    with open(newfile,'w') as f:
        f.write(query)
    print('检索式生成完成')
def Updatewos_ACS(df_all):
    try:
        df_all['doi'] = df_all['DOI'].astype('str')
        df_all['UT'] = df_all['UT (Unique ID)'].astype('str')
    except:
        df_all['doi'] = df_all['doi'].astype('str')
        df_all['UT'] = df_all['UT'].astype('str')
    conn = pymssql.connect(host=key.sql.host, port=key.sql.port ,user=key.sql.username, password=key.sql.password)
    # 获取cursor对象
    cs1 = conn.cursor()
    for i in range(len(df_all)):
        query = "update [BP_Extension].[dbo].[ACS_Edsug] set UT = '" + df_all.iloc[i]['UT'] +"' where doi = '" + df_all.iloc[i]['doi'] +"'"
        cs1.execute(query)
    # 提交之前的操作，如果之前已经执行多次的execute，那么就都进行提交
    conn.commit()
    # 关闭cursor对象
    cs1.close()
    # 关闭connection对象
    conn.close()
    # 创建connection连接
    print('完成更新，共',len(df_all),'条记录')
def Check_ACS():
    conn = pymssql.connect(host=key.sql.host, port=key.sql.port ,user=key.sql.username, password=key.sql.password)
    query = "select doi,title,UT from [BP_Extension].[dbo].[ACS_Edsug] where UT = '' or UT = 'nan'"
    df_utnull = pd.read_sql(query,con=conn)
    # 提交之前的操作，如果之前已经执行多次的execute，那么就都进行提交
    # 关闭connection对象
    conn.close()
    # 创建connection连接
    df_utnull = df_utnull[['doi','UT','title']]
    print('仍有',len(df_utnull),'条空记录请检查acs_null.csv文件')
    df_utnull.to_csv(r'./acs_null.csv',index = False)

def Concat_xls(files):
    files = r"./" + files + "/*.xls"
    df_all = pd.DataFrame()
    lst = glob.glob(files)
    for li in lst:
        dfi = pd.read_excel(li)
        df_all = pd.concat([df_all,dfi])
    return df_all

In [25]:
def Openwos():
    brower = webdriver.Chrome()
    #检索结果页地址
    url = "https://www.webofscience.com/wos/alldb/basic-search"
    brower.get(url)
    return brower

def Crul(brower):
    time.sleep(3)
    #设置爬取页面
    #handles = brower.window_handles
    #brower.switch_to_window(handles[3])
    num = brower.find_element_by_xpath('/html/body/app-wos/div/div/main/div/app-input-route/app-base-summary-component/app-search-friendly-display/div[1]/app-general-search-friendly-display/h1/span').text.replace(',','')
    page1000 = int(int(num)/1000)+1
    count = 0
    try:
        brower.find_element_by_xpath('//*[@id="pendo-close-guide-8fdced48"]').click()
        time.sleep(2)
        brower.find_element_by_xpath('//*[@id="pendo-close-guide-dc656865"]').click()
        time.sleep(1)
    except:
        1
    for i in range(page1000):
    #也可设置为从第n开始
    #for i in range(n, 120):
        count += 1
        if count ==60:
            time.sleep(30)
        #如果爬取超过90页，就休眠120S，可以自行设置页数
        print("第" + str(i+1) + "次")     
        
        time.sleep(7)
        brower.find_element_by_xpath('//*[@id="snRecListTop"]/app-export-menu/div/button').click()
        time.sleep(1)
        brower.find_element_by_xpath('//*[@id="exportToExcelButton"]').click()
        
        #点击recordfrom

        time.sleep(1)
        brower.find_element_by_xpath('//*[@id="radio3"]/label/span[2]').click() 
        time.sleep(1)

        start = brower.find_element_by_xpath('/html/body/app-wos/div/div/main/app-input-route/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/div/fieldset/mat-radio-group/div[3]/mat-form-field[1]/div/div[1]/div/input')
        end = brower.find_element_by_xpath('/html/body/app-wos/div/div/main/app-input-route/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/div/fieldset/mat-radio-group/div[3]/mat-form-field[2]/div/div[1]/div/input')
        start.clear()
        end.clear()
        if i != page1000-1:
            start_no = 1 + i*1000
            end_no =  1000 + i*1000
        else:
            start_no = end_no + 1
            end_no = num
        start.send_keys(start_no)
        end.send_keys(end_no)
        time.sleep(1)

        #选择record content
        brower.find_element_by_xpath('/html/body/app-wos/div/div/main/app-input-route/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/div/div[1]/wos-select/button').click()
        brower.find_element_by_xpath('//*[@id="global-select"]/div/div[2]/div[1]').click()

        brower.find_element_by_xpath('/html/body/app-wos/div/div/main/app-input-route/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/div/div[2]/button[1]/span[1]/span').click()
        time.sleep(7)
    print('下载完成')

In [ ]:
def ACS():
    starttime = time.time()
    lst_dic = ACSdownload()
    df = ACSprocess(lst_dic)
    QueryGener(file)
    print('finish')
    endtime = time.time()
    print('总耗时', endtime - starttime, 'S')

In [13]:
lst_dic = ACS_updatedownload(key)

1
记录数 20
2
记录数 40
3
记录数 60
4
记录数 80
5
记录数 100
6
记录数 120
7
记录数 140
8
记录数 160
9
记录数 180
10
记录数 200
11
记录数 220
12
记录数 240
13
记录数 260
14
记录数 280
15
记录数 300
16
记录数 320
17
记录数 340
18
记录数 360
19
记录数 380
20
记录数 400
21
记录数 420
22
记录数 440
23
记录数 460
24
记录数 480
25
记录数 500
26
记录数 520
27
记录数 540
28
记录数 560
29
记录数 580
30
记录数 600
31
记录数 620
32
记录数 640
33
记录数 660
34
记录数 680
35
记录数 700
36
记录数 720
37
记录数 740
38
记录数 760
39
记录数 780
40
记录数 800
41
记录数 820
42
记录数 840
43
记录数 860
44
记录数 880
45
记录数 900
46
记录数 920
47
记录数 940
48
记录数 960
49
记录数 980
50
记录数 1000
51
记录数 1020
52
记录数 1040
53
记录数 1060
54
记录数 1080
55
记录数 1100
56
记录数 1120
57
记录数 1140
58
记录数 1160
59
记录数 1180
60
记录数 1200
61
记录数 1220
62
记录数 1240
63
记录数 1260
64
记录数 1280
65
记录数 1300
66
记录数 1320
67
记录数 1340
68
记录数 1360
69
记录数 1380
70
记录数 1400
71
记录数 1420
72
记录数 1440
73
记录数 1460
74
记录数 1480
75
记录数 1500
76
记录数 1520
77
记录数 1540
78
记录数 1560
79
记录数 1580
80
记录数 1600
81
记录数 1620
82
记录数 1640
83
记录数 1660
84
记录数 1680
85
记录数 1700
86
记录数 1720
87
记录数 1740
88
记录数 1760
89
记录数

In [16]:
df = ACSprocess(lst_dic)
df['ACS_tag'] = 1
df['UT'] = ''

开始数据处理
数据处理完成


In [17]:
CASuploadData(df,key)

In [19]:
QueryGener_ACS(key)

检索式生成完成


In [33]:
Check()

仍有 2020 条空记录请检查acs_null.csv文件


In [22]:
borwer = Openwos()

In [24]:
Crul(borwer)

第1次
第2次
第3次


In [31]:
df_all = Concat_xls('ACS')
df_all[df_all.duplicated('DOI',keep = False)][['DOI','UT (Unique ID)','Article Title']]


,DOI,UT (Unique ID),Article Title
34,10.1021/acs.jpca.1c05194,WOS:000672731100001,A Black Scientist's Retrospective of His Life ...
35,10.1021/acs.jpca.1c05194,MEDLINE:34259000,A Black Scientist's Retrospective of His Life ...
36,10.1021/acs.jpca.1c05194,MEDLINE:34181417,A Black Scientist's Retrospective of His Life ...
173,10.1021/acsnano.8b00047,WOS:000436216800002,Fire-resistant nano-wallpaper sounds the alarm
226,10.1021/acsnano.8b00047,WOS:000431088200011,Fire Alarm Wallpaper Based on Fire-Resistant H...
425,10.1021/acsinfecdis.7b00064,WOS:000417405100003,Sugar in Breast Milk kills Bacteria
457,10.1021/acsinfecdis.7b00064,WOS:000407771400006,Human Milk Oligosaccharides Exhibit Antimicrob...


In [32]:
df_all = df_all[df_all['UT (Unique ID)'] != 'WOS:000436216800002']
df_all = df_all[df_all['UT (Unique ID)'] != 'WOS:000417405100003']
df_all = df_all[df_all['UT (Unique ID)'] != 'MEDLINE:34259000']
df_all = df_all[df_all['UT (Unique ID)'] != 'MEDLINE:34181417']

In [34]:
Updatewos(df_all)

完成更新，共 1998 条记录


In [35]:
Check()

仍有 27 条空记录请检查acs_null.csv文件


In [36]:
#增加UT后进行
df = pd.read_csv(r'./acs_null.csv')
Updatewos(df)
Check()

完成更新，共 27 条记录
仍有 23 条空记录请检查acs_null.csv文件


# 进行数据更新

In [37]:
lst_dic = ACS_updatedownload(key)

1
记录数 0
爬取完毕
0


In [38]:
df = ACSprocess(lst_dic)
if len(df) >0:
    df['ACS_tag'] = 1
    df['UT'] = ''
    uploadData(df,key)

无处理记录


In [40]:
QueryGener_ACS(key)

检索式生成完成


In [ ]:
df = pd.read_csv(r'./acs_null.csv')
Updatewos(df)